In [29]:
# Import libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
import seaborn as sns
import random
import scipy
from scipy.stats import chi2

In [78]:
# Read the data
df = pd.read_csv('data/MC3_all.csv', sep='\t')

# Filter by cancer type
df=df[df['type']=='LUAD']

# Formatting the gender
df['gender'] = df['gender'].replace(to_replace='FEMALE', value=0) # FEMALE -> 0
df['gender'] = df['gender'].replace(to_replace='MALE', value=1) # MALE -> 1

df['Late stage'] = np.where(df['ajcc_pathologic_tumor_stage']== 'Stage IV', 1, 0)    

df

,bcr_patient_barcode,X,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction,leuk.frac,Late stage
90,TCGA-05-4244,5803,LUAD,70.0,1,[Not Available],Stage IV,[Not Applicable],Lung Adenocarcinoma,[Not Available],...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.246926,1
91,TCGA-05-4249,5805,LUAD,67.0,1,[Not Available],Stage IB,[Not Applicable],Lung Adenocarcinoma,[Not Available],...,1523.0,0.0,1523.0,NaN,NaN,0.0,1523.0,NaN,0.251681,0
92,TCGA-05-4250,5806,LUAD,79.0,0,[Not Available],Stage IIIA,[Not Applicable],Lung Adenocarcinoma,[Not Available],...,121.0,NaN,121.0,NaN,NaN,0.0,121.0,NaN,0.238785,0
93,TCGA-05-4382,5807,LUAD,68.0,1,[Not Available],Stage IB,[Not Applicable],Lung Adenocarcinoma,[Not Available],...,607.0,0.0,607.0,1.0,334.0,1.0,334.0,NaN,0.460245,0
94,TCGA-05-4384,5808,LUAD,66.0,1,[Not Available],Stage IIIA,[Not Applicable],Lung Adenocarcinoma,[Not Available],...,426.0,0.0,426.0,NaN,NaN,1.0,183.0,NaN,0.199447,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9137,TCGA-NJ-A55O,6320,LUAD,56.0,0,WHITE,Stage IIA,[Not Applicable],Lung Adenocarcinoma,[Not Available],...,13.0,0.0,13.0,NaN,NaN,0.0,13.0,NaN,0.279332,0
9138,TCGA-NJ-A55R,6321,LUAD,67.0,1,WHITE,Stage IA,[Not Applicable],Lung Adenocarcinoma,[Not Available],...,603.0,0.0,603.0,NaN,NaN,0.0,603.0,NaN,0.345369,0
9139,TCGA-NJ-A7XG,6322,LUAD,49.0,1,BLACK OR AFRICAN AMERICAN,Stage IIIA,[Not Applicable],Lung Adenocarcinoma,[Not Available],...,617.0,0.0,617.0,NaN,NaN,0.0,617.0,NaN,0.122267,0
9147,TCGA-O1-A52J,6323,LUAD,74.0,0,WHITE,Stage IA,[Not Applicable],Lung Adenocarcinoma,[Not Available],...,1798.0,1.0,1798.0,NaN,NaN,1.0,898.0,NaN,0.283192,0


In [79]:
# Reduced data frame with the desired parameters
cox_df=df[['leuk.frac', 'gender', 'age_at_initial_pathologic_diagnosis','OS.time','OS',]] 

# Removing rows with NAs
cox_df = cox_df[cox_df['OS.time'].notna()]
cox_df = cox_df[cox_df['age_at_initial_pathologic_diagnosis'].notna()]

# Fitting the model and printing the results
cph = CoxPHFitter()
cph.fit(cox_df, duration_col = 'OS.time', event_col='OS')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 521 total observations, 334 right-censored observations>
             duration col = 'OS.time'
                event col = 'OS'
      baseline estimation = breslow
   number of observations = 521
number of events observed = 187
   partial log-likelihood = -1002.53
         time fit was run = 2023-03-15 14:53:22 UTC

---
                                      coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                   
leuk.frac                            -0.76       0.47       0.56            -1.85             0.34                 0.16                 1.40
gender                                0.12       1.13       0.15            -0.17             0.41                 0.84                 1.50
age_at_initial_pathologic_diagnosis   0.01       1.01       0.01            -0.01             0.02                 0.99                 1.02

                                      cmp to     z    p   -log2(p)
covariate                                                         
leuk.frac                               0.00 -1.36 0.17       2.52
gender                                  0.00  0.81 0.42       1.25
age_at_initial_pathologic_diagnosis     0.00  0.87 0.38       1.38
---
Concordance = 0.56
Partial AIC = 2011.07
log-likelihood ratio test = 3.52 on 3 df
-log2(p) of ll-ratio test = 1.65

In [80]:
# Reduced data frame with the desired parameters
cox_df=df[['leuk.frac', 'gender', 'age_at_initial_pathologic_diagnosis','PFI.time','PFI']]

# Removing rows with NAs
cox_df = cox_df[cox_df['PFI.time'].notna()]
cox_df = cox_df[cox_df['age_at_initial_pathologic_diagnosis'].notna()]

# Fitting the model and printing the results
cph = CoxPHFitter()
cph.fit(cox_df, duration_col = 'PFI.time', event_col='PFI')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 521 total observations, 304 right-censored observations>
             duration col = 'PFI.time'
                event col = 'PFI'
      baseline estimation = breslow
   number of observations = 521
number of events observed = 217
   partial log-likelihood = -1200.52
         time fit was run = 2023-03-15 14:53:23 UTC

---
                                      coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                   
leuk.frac                            -0.10       0.91       0.53            -1.13             0.93                 0.32                 2.54
gender                                0.19       1.21       0.14            -0.08             0.46                 0.93                 1.58
age_at_initial_pathologic_diagnosis  -0.00       1.00       0.01            -0.02             0.01                 0.98                 1.01

                                      cmp to     z    p   -log2(p)
covariate                                                         
leuk.frac                               0.00 -0.18 0.85       0.23
gender                                  0.00  1.40 0.16       2.63
age_at_initial_pathologic_diagnosis     0.00 -0.37 0.71       0.49
---
Concordance = 0.55
Partial AIC = 2407.04
log-likelihood ratio test = 2.21 on 3 df
-log2(p) of ll-ratio test = 0.92